In [1]:
import pandas as pd
from dataworks import DataWorks
dws = DataWorks()



In [80]:
def load_choice_file(path, field, name):
    """读取choice数据终端导出的数据文件.
        Choiced导出文件格式对应的处理规则. 
        - 原文件第一行为“宏观数据“或类似内容,但read_excel方法未加载该行内容
        - 数据的第一行作为指标标题
        - 第一列作为日期
        - 前六行和最后一行都不是数据内容
        - 剔除“日期”字段为空的行和其他非数据内容（标识数据来源的文字内容）
        
        Choice文件导出注意事项:
        - 导出字段选择:指标名称、频率、单位、来源
        - 日期排序:降序
        - 图形设置:不导出图形
        - 勾选“使用函数方式导出”
    Args:
        file_path (str): Choice导出文件(.xlsx)的绝对路径

    Returns:
        dataframe: 将Choice导出文件内容加载到dataframe,并返回
    """
    if path=='':
        return None
    df = pd.read_excel(path)
    df.columns = df.iloc[0]
    df.rename(columns={df.columns[0]: 'date'}, inplace=True)
    df = df.iloc[6:]
    if isinstance(field, list):
        df[name] = df[field[0]] + df[field[1]]
        df = df[['date', name]]
    else:
        df.rename(columns={field: name}, inplace=True)
        df = df[['date', name]]
        
    # df.reset_index(drop=True, inplace=True)
    # df.set_index('date', inplace=True)
    df.dropna(axis=0, subset=['date'], inplace=True)
    df = df[df['date'] != '数据来源：东方财富Choice数据']
    df['date'] = pd.to_datetime(df['date'])
    return df

In [3]:
import akshare as ak
trade_date = ak.tool_trade_date_hist_sina()['trade_date']
trade_date.iloc[0]

datetime.date(1990, 12, 19)

In [81]:
config = dws.get_variety_setting('setting/variety.json')
variety_config = config['RB']
combined_df = None
for data_type, data_info in variety_config['DataIndex'].items():
    # 从文件中加载指定字段的数据
    data_df = load_choice_file(data_info['Path'], data_info['Field'], data_info['Name'])
    
    # 将数据添加到combined_df中
    # combined_df = pd.concat([combined_df, data_df], axis=1)
    combined_df = pd.merge(combined_df, data_df, on='date', how='outer') if combined_df is not None else data_df
# combined_df.dropna(axis=0, subset=['date'], inplace=True)

In [2]:
import pandas as pd
df_basis = dws.get_data_by_sql('SELECT * FROM basis')
df_basis = df_basis[['date', 'dom_basis']]

In [3]:
df_basis['date'] = pd.to_datetime(df_basis['date'])
ts = df_basis.set_index('date')['dom_basis']

In [19]:
import re

def extract_variables(format_str):
    # 正则表达式模式，匹配非空字符（即变量）
    pattern = r'\{.*?\}'
    # 使用正则表达式查找所有匹配的变量名
    variables = re.findall(pattern, format_str)
    # 去除花括号并返回结果
    return [var[1:-1] for var in variables]

# 示例四则运算公式字符串
format_str = "a * b / (c % d) + e - f ** g"
format_str2 = "{abc:efg} + {137a:mn}"

# 提取并打印变量列表
variables_list = extract_variables(format_str2)
print("Extracted variables:", variables_list)

Extracted variables: ['abc:efg', '137a:mn']


In [12]:
import re
def extract_variables(format_str):
    """
    从格式字符串中提取变量名，包括可能包含冒号 ":" 的变量。

    参数:
    format_str: 一个包含变量名的字符串。例如："abc:efg + 137a:mn + name + age"

    返回值:
    一个包含所有提取出的变量名（可能含有冒号）的列表。例如：['abc:efg', '137a:mn', 'name', 'age']。
    """
    # 定义用于匹配可能包含冒号的变量名的正则表达式模式
    variable_pattern = r'\w[\w:()%]*'
    # 使用正则表达式查找所有匹配的变量名
    variables = re.findall(variable_pattern, format_str)
    return variables  # 直接返回找到的变量名列表，无需额外处理

format_str2 = "库存:热卷(板)"

# 提取并打印变量列表
variables_list = extract_variables(format_str2)
print("Extracted variables:", variables_list)

Extracted variables: ['库存:热卷(板)']


In [ ]:
import dash_mantine_components as dmc
from dash import Dash, html, Input, Output, State
from dash_iconify import DashIconify

app = Dash(__name__)

SIDEBAR_WIDTH = 200
quick_access = html.Div(
    style={"width": SIDEBAR_WIDTH, "height": "auto"},
    children=[
        dmc.NavLink(
            label="板块品种",
            icon=DashIconify(icon="akar-icons:link-chain", width=24),
            childrenOffset=28,
            children=[
                dmc.NavLink(label="First child link"),
                dmc.NavLink(label="Second child link"),
            ],
            variant="subtle",
            opened=True,
            active=True,
        ),
        dmc.NavLink(
            label="自选品种",
            icon=DashIconify(icon="fluent-mdl2:favorite-list", width=24),
            childrenOffset=28,            
            children=[
                dmc.NavLink(label="First child link", variant="subtle"),
                dmc.NavLink(label="Second child link"),
                dmc.NavLink(label="Third child link"),
            ],
            variant="subtle",
            opened=True,
            active=True,
        ),
    ],
)

accordion_methods = dmc.Accordion(
    # style={"width": SIDEBAR_WIDTH, "height": "auto"},
    # chevron=DashIconify(icon="ant-design:plus-outlined"),
    # disableChevronRotation=True,
    chevronPosition="left",
    chevron=False,
    value="basis_analysis",
    # variant="filled",
    children=[
        dmc.AccordionItem(
            [
                dmc.AccordionControl("基本面分析"),
                dmc.AccordionPanel("基差-库存-利润模型分析", style={"fontSize": "13px", "color": "gray"}),
            ],
            value="basis_analysis",
            
        ),
        dmc.AccordionItem(
            [
                dmc.AccordionControl("周期性分析"),
                dmc.AccordionPanel("基差、库存、持仓等周期性规律，基于ARIMA模型的AI分析", style={"fontSize": "13px", "color": "gray"}),
            ],
            value="cycle_analysis",
        ),
        dmc.AccordionItem(
            [
                dmc.AccordionControl("主成分分析"),
                dmc.AccordionPanel("利用PCA进行主成分分析，指标影响可视化", style={"fontSize": "13px", "color": "gray"}),
            ],
            value="pda_analysis",
        ),
        dmc.AccordionItem(
            [
                dmc.AccordionControl("套利分析"),
                dmc.AccordionPanel("跨期、跨品种套利分析", style={"fontSize": "13px", "color": "gray"}),
            ],
            value="cross_analysis",
        ),
        dmc.AccordionItem(
            [
                dmc.AccordionControl("强化学习"),
                dmc.AccordionPanel("利用DRL算法进行AI分析", style={"fontSize": "13px", "color": "gray"}),
            ],
            value="drl_analysis",
        ),
        dmc.AccordionItem(
            [
                dmc.AccordionControl("技术分析"),
                dmc.AccordionPanel("技术指标、缠论分析", style={"fontSize": "13px", "color": "gray"}),
            ],
            value="tech_analysis",
        ),                        
   ],
)

methods = html.Div(
    style={"width": 200, "height": "auto"},
    children=[
        dmc.NavLink(
            label="基本面分析",
            description="基差-库存-利润模型分析",
            icon=DashIconify(icon="akar-icons:link-chain", width=24),            
            childrenOffset=28,
            variant="contained",
        ),
        dmc.NavLink(
            label="周期性分析",
            description="基差、库存、持仓等周期性规律，基于ARIMA模型的AI分析",
            icon=DashIconify(icon="fluent-mdl2:favorite-list", width=24),
            childrenOffset=28,            
            variant="contained",
        ),
        dmc.NavLink(
            label="主成分分析",
            description="利用PCA进行主成分分析，指标影响可视化",
            icon=DashIconify(icon="fluent-mdl2:favorite-list", width=24),
            childrenOffset=28,            
            variant="contained",
        ),        
        dmc.NavLink(
            label="套利分析",
            description="跨期、跨品种套利分析",
            icon=DashIconify(icon="fluent-mdl2:favorite-list", width=24),
            childrenOffset=28,            
            variant="contained",
        ),               
        dmc.NavLink(
            label="强化学习分析",
            description="利用DRL算法进行AI分析",
            icon=DashIconify(icon="fluent-mdl2:favorite-list", width=24),
            childrenOffset=28,            
            variant="contained",
        ),            
    ],
)

sidebar_tabs =dmc.Tabs(
    [
        dmc.TabsList(
            [
                dmc.Tab("分析", value="analyze"),
                dmc.Tab("板块", value="chain"),
                dmc.Tab("自选", value="favorite"),
            ]
        ),
        dmc.TabsPanel(accordion_methods, value="analyze"),
        dmc.TabsPanel(quick_access, value="chain"),
        dmc.TabsPanel("Settings tab content", value="favorite"),
    ],
    value="analyze",
)

paper = dmc.Paper(
    [
        sidebar_tabs
    ],
    shadow="sm",
    p="xl",
    withBorder=True,
)

header = dmc.Header(
    [        
        dmc.Paper(
            dmc.Container(
                children=[
                    dmc.Group(
                        children=[
                            DashIconify(icon="simple-icons:bitcoincash", width=24, color="gray"),
                            dmc.Text("Futures Nexus", color="gray"),
                            dmc.Space(w=60),
                            dmc.TextInput(placeholder="Input the various name"),
                            dmc.Button("Search", variant="outline", leftIcon=DashIconify(icon="wpf:search", width=20)),
                            dmc.Space(w=60),
                            dmc.Menu(                    
                                [
                                    dmc.MenuTarget(dmc.Button("金属", variant="subtle", leftIcon=DashIconify(icon="icon-park-outline:heavy-metal", width=24))),
                                    dmc.MenuDropdown(
                                        children=[                                    
                                            dmc.MenuItem("钢铁产业链", icon=DashIconify(icon="tabler:ironing-steam", width=20)),
                                            dmc.MenuItem("贵金属产业链", icon=DashIconify(icon="ant-design:gold-outlined", width=20)),
                                            dmc.MenuItem("铜产业链", icon=DashIconify(icon="mingcute:copper-coin-line", width=20)),
                                            dmc.MenuItem("铝产业链", icon=DashIconify(icon="file-icons:silverstripe", width=20)),
                                        ],
                                    ),                       
                                ],
                            ),
                            dmc.Menu(                    
                                [
                                    dmc.MenuTarget(dmc.Button("能源", variant="subtle", leftIcon=DashIconify(icon="material-symbols:energy-savings-leaf-outline", width=24))),
                                    dmc.MenuDropdown(
                                        children=[                                    
                                            dmc.MenuItem("动力煤产业链", icon=DashIconify(icon="mdi:charcoal-outline", width=20)),
                                            dmc.MenuItem("石油产业链", icon=DashIconify(icon="lets-icons:oil", width=20)),
                                            dmc.MenuItem("原油产业链", icon=DashIconify(icon="mdi:boil-point-outline", width=20)),
                                        ],
                                    ),                       
                                ],
                            ),                    
                            dmc.Menu(                    
                                [
                                    dmc.MenuTarget(dmc.Button("化工", variant="subtle", leftIcon=DashIconify(icon="mdi:chemical-weapon", width=24))),
                                    dmc.MenuDropdown(
                                        children=[                                    
                                            dmc.MenuItem("化工产业链", icon=DashIconify(icon="healthicons:chemical-burn-outline", width=20)),
                                            dmc.MenuItem("沥青产业链", icon=DashIconify(icon="arcticons:asphalt-8", width=20)),
                                            dmc.MenuItem("橡胶产业链", icon=DashIconify(icon="tabler:rubber-stamp", width=20)),
                                        ],
                                    ),                       
                                ],
                            ),          
                            dmc.Menu(                    
                                [
                                    dmc.MenuTarget(dmc.Button("农产品", variant="subtle", leftIcon=DashIconify(icon="ic:outline-agriculture", width=24))),
                                    dmc.MenuDropdown(
                                        children=[                                    
                                            dmc.MenuItem("菜籽油产业链", icon=DashIconify(icon="mdi:seedling-outline", width=20)),
                                            dmc.MenuItem("大豆产业链", icon=DashIconify(icon="ph:coffee-bean-bold", width=20)),
                                            dmc.MenuItem("糖产业链", icon=DashIconify(icon="ep:sugar", width=20)),
                                            dmc.MenuItem("小麦产业链", icon=DashIconify(icon="lucide:wheat", width=20)),
                                            dmc.MenuItem("玉米产业链", icon=DashIconify(icon="tdesign:corn", width=20)),
                                            dmc.MenuItem("棕榈油产业链", icon=DashIconify(icon="ph:tree-palm", width=20)),
                                            dmc.MenuItem("生猪产业链", icon=DashIconify(icon="clarity:piggy-bank-line", width=20)),
                                        ],
                                    ),                       
                                ],
                            ),                                   
                            dmc.Menu(                    
                                [
                                    dmc.MenuTarget(dmc.Button("自选", variant="subtle", leftIcon=DashIconify(icon="mdi:favorite-settings-outline", width=24))),               
                                ],
                            ),                       
                            dmc.Menu(                    
                                [
                                    dmc.MenuTarget(dmc.Button("设置", variant="subtle", leftIcon=DashIconify(icon="icon-park-outline:config", width=24))),
                                    dmc.MenuDropdown(
                                        children=[                                    
                                            dmc.MenuItem("数据管理", icon=DashIconify(icon="icon-park-outline:data-switching", width=20)),
                                            dmc.MenuItem("品种管理", icon=DashIconify(icon="carbon:data-class", width=20)),
                                            dmc.MenuItem("通用设置", icon=DashIconify(icon="grommet-icons:configure", width=20)),
                                        ],
                                    ),                       
                                ],
                            ),                                                       
                        ],
                        # position="apart",
                    ),
                ],
                fluid=True,
                # style={"height": "100px"},
            ),
            shadow="sm",
            radius="xs",
            p="xs",
            withBorder=True,
        )        
    ], 
    height="auto", 
    # style={"backgroundColor": "#9c86e2"}
)

footer = dmc.Footer(
    height=30, children=[dmc.Text("Company Logo")]
)

style = {
    "border": f"1px solid {dmc.theme.DEFAULT_COLORS['indigo'][4]}",
    "textAlign": "center",
}

seg_variety = dmc.SegmentedControl(
    id="segmented-variety-switcher",
    value="market_overview",    
    data=[
        {"value": "market_overview", "label": "市场全景"},
        {"value": "ng", "label": "螺纹钢"},
        {"value": "svelte", "label": "铁矿石"},
        {"value": "new", "label": "+"},
    ],
    # mt=10,
    color="gray",
    style={"backgroundColor": "white"}
),

main_context = html.Div(dmc.Grid(
    children=[        
        dmc.Col(paper, span=2),
        dmc.Col(
            dmc.Stack(
                children=[
                    dmc.Grid(
                        [
                            dmc.Col(seg_variety, span="auto"),
                            dmc.Col(
                                dmc.ActionIcon(DashIconify(icon="carbon:close-outline"), id="button-remove-tab", variant="subtle", color="blue", size=36, radius="xs"),                    
                                xs=0.35),
                            dmc.Col(                  
                                dmc.ActionIcon(DashIconify(icon="tabler:settings"), id="button-methond-config", variant="subtle", color="blue", size=36, radius="xs"),
                                xs=0.35),           
                        ]
                    ),
                    dmc.Divider(variant="solid"),
                ],
                p=5,
                spacing=0,
            ),
        span="auto"),
    ],
    gutter="5",
))

app.layout = html.Div(
    [
        dmc.Stack(
            children=[
                header,
                main_context,
                # footer,
            ],
            spacing=4,
        )
    ]
)

@app.callback(
    Output("segmented-variety-switcher", "data"),
    Output("segmented-variety-switcher", "value"),
    Input("segmented-variety-switcher", "value"),
    State("segmented-variety-switcher", "data"),
)
def update_variety(value, data):    
    if value == "new":
        new_value = "untiltled"
        new_item = {"value": new_value, "label": "Untiltled"}
        data.insert(-1, new_item)
        return data, new_value
    else:
        return data, value
    
@app.callback(
    Output("segmented-variety-switcher", "data", allow_duplicate=True),
    Output("segmented-variety-switcher", "value", allow_duplicate=True),
    Input("button-remove-tab", "n_clicks"),
    State("segmented-variety-switcher", "value"),
    State("segmented-variety-switcher", "data"),
    prevent_initial_call=True,
)
def remove_tab(n_clicks, value, data):
    if len(data) <=1:
        return data, value
    select_index = [index for index, item in enumerate(data) if value not in item['value']]
    del data[select_index[0]]
    active_value = data[0]['value']
    return data, active_value

if __name__ == "__main__":
    app.run_server(port=8051)

In [1]:
import json
import dataworks as dw
dws = dw.DataWorks()

In [2]:
def load_choice_file(file_path):
    """读取choice数据终端导出的数据文件.
        Choiced导出文件格式对应的处理规则. 
        - 原文件第一行为“宏观数据“或类似内容,但read_excel方法未加载该行内容
        - 数据的第一行作为指标标题
        - 第一列作为日期
        - 前六行和最后一行都不是数据内容
        - 剔除“日期”字段为空的行和其他非数据内容（标识数据来源的文字内容）
        
        Choice文件导出注意事项:
        - 导出字段选择:指标名称、频率、单位、来源
        - 日期排序:降序
        - 图形设置:不导出图形
        - 勾选“使用函数方式导出”
    Args:
        file_path (str): Choice导出文件(.xlsx)的绝对路径

    Returns:
        dataframe: 将Choice导出文件内容加载到dataframe,并返回
    """
    if file_path=='':
        return None
    df = pd.read_excel(file_path)
    df.columns = df.iloc[0]
    df.rename(columns={df.columns[0]: 'date'}, inplace=True)
    df = df[6:]
    df.reset_index(drop=True, inplace=True)
    df.dropna(axis=0, subset=['date'], inplace=True)
    df = df[df['date'] != '数据来源：东方财富Choice数据']
    df['date'] = pd.to_datetime(df['date'])
    return df

In [3]:
common_json = 'setting/common.json'
variety_json = 'setting/variety.json'
symbol_id = "RB"

with open(common_json, encoding='utf-8') as common_file: 
    symbol_dataindex_setting = json.load(common_file)['DataIndex']
with open(variety_json, encoding='utf-8') as variety_file:
    variety_setting = json.load(variety_file)[symbol_id]     
variety_setting['DataIndex'] = {**symbol_dataindex_setting, **variety_setting['DataIndex']} if 'DataIndex' in variety_setting else symbol_dataindex_setting
symbol_setting = variety_setting


In [4]:
import pandas as pd
import re
from asteval import Interpreter

def extract_variables(format_str):
    """从格式字符串中提取变量名"""
    # 正则表达式模式，匹配非空字符（即变量）
    variable_pattern = r'\w[\w:()%]*'
    # 使用正则表达式查找所有匹配的变量名
    variables = re.findall(variable_pattern, format_str)
    return variables  # 直接返回找到的变量名列表，无需额外处理

column_dict= {}
data_frames = []
data_index = symbol_setting['DataIndex']

# dws = gs.dataworks
dws = dw.DataWorks()
for key, value_items in data_index.items():
    # 按照配置文件中的DataFrame键值，将同类内容合并到同一张表中
    df_name = value_items['DataFrame']            
    fields = value_items['Field']
    variables_list = extract_variables(fields)
    if df_name in locals():        
        # 键值是独立字段的，列名修改为key
        if len(variables_list)==1:
            locals()[df_name].rename(columns={variables_list[0]:key}, inplace=True)
    else:
        # 未加载的数据，整表读取到df_name中
        data_source = value_items['Source']
        if data_source=='Choice':
            locals()[df_name] = load_choice_file(value_items['Path'])
        elif data_source=='SQLite':
            locals()[df_name] = dws.get_data_by_symbol(value_items['Path'], symbol_id, '*')
            locals()[df_name]['date'] = pd.to_datetime(locals()[df_name]['date'])
        else:
            continue
        column_dict[df_name] = ['date']
    df = locals()[df_name]            
    if len(variables_list)==1:
        # Field是独立字段的，列名修改为key
        df.rename(columns={variables_list[0]:key}, inplace=True)
        column_dict[df_name].append(key)
    else:                
        # Field是公式表达的，进行解析计算
        aeval = Interpreter()       
        for var in variables_list:                    
            safe_var = re.sub(r'[0-9:]', '', var)
            df.rename(columns={var:safe_var}, inplace=True)                                     
            aeval.symtable[safe_var] = df[safe_var]
        safe_fields = re.sub(r'[0-9:]', '', fields)
        df[key] = aeval.eval(safe_fields)
        column_dict[df_name].append(key)
    # 根据配置中指定的填充方式填充缺失值
    if 'FillNa' in value_items:
        fill_na = value_items['FillNa']
        print(f"symbol: {symbol_id}, key: {key}")
        if fill_na=='Forward':
            df[key] = df[key].ffill()
        elif fill_na=='Backward':
            df[key] = df[key].bfill()
        # elif fill_na=='Interpolate':

symbol: RB, key: 需求
symbol: RB, key: 产量
symbol: RB, key: 开工率
symbol: RB, key: 产能利用率
symbol: RB, key: 库存


/var/folders/hc/6dqjp7bs58s2pcb47s4xfsf80000gn/T/ipykernel_28136/3172956327.py:59: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[key] = df[key].ffill()
/var/folders/hc/6dqjp7bs58s2pcb47s4xfsf80000gn/T/ipykernel_28136/3172956327.py:59: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[key] = df[key].ffill()


In [5]:
from functools import reduce
for df_key in column_dict:
    df = locals()[df_key]
    df = df[column_dict[df_key]]
    data_frames.append(df)
symbol_data = reduce(lambda left,right: pd.merge(left,right,on='date', how='outer'), data_frames)
# if "利润" in self.symbol_setting:
#     for key, value in enumerate(self.symbol_setting['利润'].items()):
#         variables_list = extract_variables(value)
#         aeval = Interpreter()       
#         for var in variables_list:                                              
#             aeval.symtable[var] = df[var]
#         self.symbol_data[key] = aeval.eval(value)

symbol_data.sort_values(by='date', ascending=True, inplace=True)
columns_to_check = symbol_data.columns[1:]
symbol_data.dropna(subset=columns_to_check, how='all', inplace=True)        

In [34]:
import akshare as ak
trade_date = ak.tool_trade_date_hist_sina()['trade_date']
trade_date = pd.to_datetime(trade_date)


In [35]:
date_column = 'date'
# 筛选出 date 在 trade_date 中的行
valid_dates_mask = symbol_data[date_column].isin(trade_date)
# 删除不属于交易日的行
new_symbol_data2 = symbol_data.loc[valid_dates_mask]

# 或者，如果您希望直接在原数据框上进行修改（inplace=True）
# symbol_data.drop(symbol_data.index[~valid_dates_mask], inplace=True)

In [ ]:
from datetime import datetime, timedelta
start_date = symbol_data['date'].iloc[0]
end_date = symbol_data['date'].iloc[-1]
interval = timedelta(days=1)  
all_date = [start_date + i * interval for i in range((end_date - start_date).days + 1)]

not_trade_date = sorted(set(all_date) - set(trade_date))

In [42]:
# 重新加载salary计算模块
import importlib
importlib.reload(variety)

<module 'variety' from 'e:\\Document\\Project\\futures_nexus\\variety.py'>

In [44]:
import variety
from variety import *
rb = SymbolData('RB')
rb.merge_data()
# symbol_data = rb._calculate_basis('主力合约收盘价')

symbol: RB, key: 库存
symbol: RB, key: 需求
symbol: RB, key: 产量
symbol: RB, key: 开工率
symbol: RB, key: 产能利用率


e:\Document\Project\futures_nexus\variety.py:207: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[key] = df[key].ffill()


,date,主力合约收盘价,主力合约结算价,主力合约持仓量,近月合约收盘价,近月合约结算价,近月合约持仓量,现货价格,基差,基差率,仓单,库存,需求,产量,开工率,产能利用率
0,2010-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4568,2024-03-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1324.80,212.38,218.53,39.02,47.91
4569,2024-03-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1289.97,246.44,211.61,39.67,46.39
4570,2024-03-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1289.97,246.44,211.61,39.67,46.39
4571,2024-03-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1289.97,246.44,211.61,39.67,46.39


In [45]:
field_list = rb.get_data_fields()

In [46]:
data_rank = rb.calculate_data_rank('主力合约',field_list,6)

data_list: ['基差率', '仓单', '库存', '需求', '产量', '开工率', '产能利用率']
data_list: ['基差率', '主力合约持仓量', '仓单', '库存', '需求', '产量', '开工率', '产能利用率']


e:\Document\Project\futures_nexus\variety.py:356: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_rank = pd.DataFrame()


In [47]:
data_rank

,date,开工率百分位,开工率分位,开工率,需求百分位,需求分位,需求,库存百分位,库存分位,库存,...,基差率,产能利用率百分位,产能利用率分位,产能利用率,产量百分位,产量分位,产量,主力合约持仓量百分位,主力合约持仓量分位,主力合约持仓量
0,2010-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,2024-03-19,0.225000,3,39.02,0.275000,4,212.38,0.983333,5,1324.80,...,NaN,0.225000,2,47.91,0.183333,2,218.53,NaN,NaN,NaN
3396,2024-03-20,0.291667,3,39.67,0.500000,4,246.44,0.916667,5,1289.97,...,NaN,0.166667,2,46.39,0.125000,2,211.61,NaN,NaN,NaN
3397,2024-03-21,0.295833,3,39.67,0.504167,4,246.44,0.912500,5,1289.97,...,NaN,0.170833,2,46.39,0.129167,2,211.61,NaN,NaN,NaN
3398,2024-03-22,0.300000,3,39.67,0.508333,5,246.44,0.908333,5,1289.97,...,NaN,0.175000,2,46.39,0.133333,2,211.61,NaN,NaN,NaN
